**Results - Regression of simulated events**

This notebook is the primary source of plots and tables for the regression part of the thesis, 
with the goal of keeping every table and figure as standardized as possible. (And who has the time to update
90 tables one by one anyway).

**Questions:**
* Descriptive statistics
    - Should descriptive statistics of the simulated data be included?\
    If so, how much? And should it be included for each fold in the k-fold cross-validation?
* Classification results
    - Breakdown of results based on event type? Single, double, close double?
    Reasonable to include in order to confirm the assumption that close doubles are the
    most difficult event type to classify correctly in simulated data
    Random state is included, so should be simple to reproduce the indices


**TODO**
* Implement reproducing the validation indices for each fold based on the random seed from config

**Handy links**
* [matplotlib-plots to latex](https://timodenk.com/blog/exporting-matplotlib-plots-to-latex/)
* [Robert's thesis df output](https://github.com/ATTPC/VAE-event-classification/blob/master/src/make_classification_table.py)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from master_scripts.data_functions import get_git_root
from master_scripts.analysis_functions import load_experiment, experiment_metrics_to_df
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

THESIS_PATH = "../../../master_thesis/"

# Pre-processed simulated data - no additional modifications
This is the basic metrics for all the models trained on simulated data.
The basic pre-processing includes formatting and min-max normalization.

## Single events

### Positions

#### Linear Regression

In [9]:
# Load linear regression experiment
lin_ex_id = "225ca879103d"
lin_ex = load_experiment(lin_ex_id)
print(lin_ex.keys())
#lin_model = tf.keras.models.load_model(repo_root + "models/" + lin_ex_id + ".h5")

lin_metrics = experiment_metrics_to_df(lin_ex)
display(lin_metrics)
lin_means = lin_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
lin_means = lin_means.rename(index={'mean': 'lin_mean', 'std': 'lin_std'})
#display(lin_means)

dict_keys(['loss', 'optimizer', 'experiment_config', 'model_type', 'experiment_name', 'experiment_id', 'datetime', 'history', 'metrics'])


,r2_score,mse,rmse,mae
fold_0,0.796910,0.014212,0.119215,0.092134
fold_1,0.802642,0.013832,0.117610,0.090416
fold_2,0.803552,0.013747,0.117249,0.089933
fold_3,0.805308,0.013653,0.116845,0.088000
fold_4,0.797909,0.014137,0.118899,0.091896


,r2_score,mse,rmse,mae
lin_mean,0.801264,0.013916,0.117964,0.090476
lin_std,0.003664,0.000246,0.001040,0.001673


#### Small dense network

In [10]:
# Load logistic regression experiment
dense_ex_id = "a3716bc3648a"
dense_ex = load_experiment(dense_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + dense_ex_id + ".h5")

dense_metrics = experiment_metrics_to_df(dense_ex)
display(dense_metrics)
dense_means = dense_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
dense_means = dense_means.rename(index={'mean': 'dense_mean', 'std': 'dense_std'})
#display(dense_means)

,r2_score,mse,rmse,mae
fold_0,0.988785,0.000785,0.028019,0.018682
fold_1,0.990244,0.000684,0.026148,0.017429
fold_2,0.990299,0.000679,0.026054,0.017095
fold_3,0.991039,0.000628,0.025068,0.016272
fold_4,0.990204,0.000685,0.026177,0.017504


,r2_score,mse,rmse,mae
dense_mean,0.990114,0.000692,0.026293,0.017396
dense_std,0.000819,0.000057,0.001069,0.000869


#### Small CNN

In [11]:
# Load logistic regression experiment
cnn_ex_id = "1cac590bf1fe"
cnn_ex = load_experiment(cnn_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + cnn_ex_id + ".h5")

cnn_metrics = experiment_metrics_to_df(cnn_ex)
display(cnn_metrics)
cnn_means = cnn_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
cnn_means = cnn_means.rename(index={'mean': 'cnn_mean', 'std': 'cnn_std'})
#display(cnn_means)

,r2_score,mse,rmse,mae
fold_0,0.996735,0.000228,0.015115,0.009773
fold_1,0.997174,0.000198,0.014074,0.008915
fold_2,0.997074,0.000205,0.014310,0.008950
fold_3,0.997386,0.000183,0.013539,0.008232
fold_4,0.997203,0.000196,0.013987,0.008577


,r2_score,mse,rmse,mae
cnn_mean,0.997114,0.000202,0.014205,0.008889
cnn_std,0.000240,0.000017,0.000580,0.000573


#### Pretrained - VGG
As an additional baseline for performance, we include a pretrained SOTA network
where trained on the ImageNet database.

Due to the size of our detector images (16x16) compared with the size the VGG network is
designed for, we cannot use all layers in the VGG network. This stems from the use of max-pooling
which effectively reduces the image size to half (8x8) each time the input is passed through such a
layer. At some point our input is too small to pass through to the rest of the network.
We therefore cut the network at the point where this becomes an issue.
Alternatively, one could possibly keep the depth but remove max-pooling layers.

In [12]:
# Load logistic regression experiment
pretrained_ex_id = "d53a2353251f"
pretrained_ex = load_experiment(pretrained_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + pretrained_ex_id + ".h5")

pretrained_metrics = experiment_metrics_to_df(pretrained_ex)
display(pretrained_metrics)
pretrained_means = pretrained_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
pretrained_means = pretrained_means.rename(index={'mean': 'pretrained_mean', 'std': 'pretrained_std'})
#display(pretrained_means)

,r2_score,mse,rmse,mae
fold_0,0.887204,0.007893,0.088841,0.056515
fold_1,0.884742,0.008078,0.089879,0.058860
fold_2,0.889443,0.007737,0.087958,0.057577
fold_3,0.891697,0.007594,0.087146,0.055001
fold_4,0.902490,0.006821,0.082592,0.051964


,r2_score,mse,rmse,mae
pretrained_mean,0.891115,0.007625,0.087283,0.055983
pretrained_std,0.006864,0.000484,0.002813,0.002656


#### Output
We use the standard deviation in the folds as an error measure, and report the mean classification f1_score.

In [17]:
all_means = pd.DataFrame(
    [
        lin_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        dense_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        cnn_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        pretrained_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']]
    ]
).rename(
    index={
        'lin_mean': 'Linear',
        'dense_mean': 'Dense',
        'cnn_mean': 'Convolutional',
        'pretrained_mean': 'Pretrained VGG16',
    }
)
display(all_means)
all_std = pd.DataFrame(
    [
        log_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        dense_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        cnn_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        pretrained_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
    ]
).rename(
    index={
        'log_std': 'Linear',
        'dense_std': 'Dense',
        'cnn_std': 'Convolutional',
        'pretrained_std': 'Pretrained VGG16',
    }
)
display(all_std)

,r2_score,mse,rmse,mae
Linear,0.926159,0.006149,0.076462,0.062479
Dense,0.931254,0.005724,0.073870,0.060016
Convolutional,0.935684,0.005355,0.071351,0.056982
Pretrained VGG16,0.934564,0.005449,0.073193,0.057032


,r2_score,mse,rmse,mae
lin_std,1.566832e+06,108287.810412,38.234234,15.665897
Dense,3.364289e-02,0.002798,0.018293,0.019395
Convolutional,3.282238e-02,0.002731,0.018183,0.019009
Pretrained VGG16,1.944682e-02,0.001616,0.010731,0.011578


In [15]:
rows = all_means.index
r2_str_array = np.zeros((1, all_means.shape[0]), dtype=object)
for i in range(all_means.shape[0]):
    r2_str_array[0, i] = r"$\underset{{\num{{+- {:.3e} }}  }}{{\num{{ {:.3g} }} }}$".format(
        all_std["r2_score"][i], all_means["r2_score"][i])
        
r2_df = pd.DataFrame(r2_str_array, columns=rows)

section_path = "chapters/results/figures/"
fname = THESIS_PATH + section_path + "regression_simulated_single_position_r2.tex"
caption = """
Mean R2-scores for regresson of simulated data using multiple models. 
Error estimates are the standard deviation in results from k-fold cross-validation 
with $K=5$ folds.
"""
label = "tab:regression-simulated-single-position-r2"
with open(fname, "w") as fp:
    pd.set_option('display.max_colwidth', -1)
    f1_df.to_latex(fp, escape=False, caption=caption, label=label, index=False)


### Energy

#### Linear regression

In [13]:
# Load linear regression experiment
lin_ex_id = "87e8f4558d97"
lin_ex = load_experiment(lin_ex_id)
print(lin_ex.keys())
#lin_model = tf.keras.models.load_model(repo_root + "models/" + lin_ex_id + ".h5")

lin_metrics = experiment_metrics_to_df(lin_ex)
display(lin_metrics)
lin_means = lin_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
lin_means = lin_means.rename(index={'mean': 'lin_mean', 'std': 'lin_std'})
#display(lin_means)

dict_keys(['loss', 'optimizer', 'experiment_config', 'model_type', 'experiment_name', 'experiment_id', 'datetime', 'history', 'metrics'])


,r2_score,mse,rmse,mae
fold_0,0.884612,0.009598,0.097968,0.084105
fold_1,0.944212,0.004654,0.068218,0.056283
fold_2,0.952773,0.003931,0.062701,0.050983
fold_3,0.961230,0.003230,0.056829,0.038072
fold_4,0.887968,0.009330,0.096594,0.082953


,r2_score,mse,rmse,mae
lin_mean,0.926159,0.006149,0.076462,0.062479
lin_std,0.036908,0.003070,0.019433,0.020329


#### Small dense network

In [14]:
# Load dense regression experiment
dense_ex_id = "4cab676db128"
dense_ex = load_experiment(dense_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + dense_ex_id + ".h5")

dense_metrics = experiment_metrics_to_df(dense_ex)
display(dense_metrics)
dense_means = dense_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
dense_means = dense_means.rename(index={'mean': 'dense_mean', 'std': 'dense_std'})
#display(dense_means)

,r2_score,mse,rmse,mae
fold_0,0.897535,0.008523,0.092319,0.078537
fold_1,0.948202,0.004321,0.065733,0.053581
fold_2,0.956246,0.003642,0.060352,0.048541
fold_3,0.962105,0.003157,0.056184,0.037481
fold_4,0.892180,0.008980,0.094761,0.081942


,r2_score,mse,rmse,mae
dense_mean,0.931254,0.005724,0.073870,0.060016
dense_std,0.033643,0.002798,0.018293,0.019395


#### Small CNN

In [15]:
# Load cnn regression experiment
cnn_ex_id = "3a91fd0e74b5"
cnn_ex = load_experiment(cnn_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + cnn_ex_id + ".h5")

cnn_metrics = experiment_metrics_to_df(cnn_ex)
display(cnn_metrics)
cnn_means = cnn_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
cnn_means = cnn_means.rename(index={'mean': 'cnn_mean', 'std': 'cnn_std'})
#display(cnn_means)

,r2_score,mse,rmse,mae
fold_0,0.907657,0.007681,0.087641,0.072885
fold_1,0.959155,0.003407,0.058371,0.044895
fold_2,0.957682,0.003523,0.059353,0.047398
fold_3,0.961143,0.003237,0.056892,0.038211
fold_4,0.892780,0.008930,0.094496,0.081520


,r2_score,mse,rmse,mae
cnn_mean,0.935684,0.005355,0.071351,0.056982
cnn_std,0.032822,0.002731,0.018183,0.019009


#### Pretrained - VGG16 

In [16]:
# Load logistic regression experiment
pretrained_ex_id = "ea8d88850f6e"
pretrained_ex = load_experiment(pretrained_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + pretrained_ex_id + ".h5")

pretrained_metrics = experiment_metrics_to_df(pretrained_ex)
display(pretrained_metrics)
pretrained_means = pretrained_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
pretrained_means = pretrained_means.rename(index={'mean': 'pretrained_mean', 'std': 'pretrained_std'})
#display(pretrained_means)

,r2_score,mse,rmse,mae
fold_0,0.904364,0.007955,0.089190,0.073793
fold_1,0.935693,0.005364,0.073242,0.059375
fold_2,0.956309,0.003637,0.060308,0.044621
fold_3,0.945288,0.004558,0.067509,0.047631
fold_4,0.931166,0.005733,0.075714,0.059741


,r2_score,mse,rmse,mae
pretrained_mean,0.934564,0.005449,0.073193,0.057032
pretrained_std,0.019447,0.001616,0.010731,0.011578


#### Output

In [ ]:
all_means = pd.DataFrame(
    [
        lin_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        dense_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        cnn_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        pretrained_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']]
    ]
).rename(
    index={
        'lin_mean': 'Linear',
        'dense_mean': 'Dense',
        'cnn_mean': 'Convolutional',
        'pretrained_mean': 'Pretrained VGG16',
    }
)
display(all_means)
all_std = pd.DataFrame(
    [
        log_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        dense_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        cnn_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        pretrained_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
    ]
).rename(
    index={
        'log_std': 'Linear',
        'dense_std': 'Dense',
        'cnn_std': 'Convolutional',
        'pretrained_std': 'Pretrained VGG16',
    }
)
display(all_std)

In [ ]:
rows = all_means.index
r2_str_array = np.zeros((1, all_means.shape[0]), dtype=object)
for i in range(all_means.shape[0]):
    r2_str_array[0, i] = r"$\underset{{\num{{+- {:.3e} }}  }}{{\num{{ {:.3g} }} }}$".format(
        all_std["r2_score"][i], all_means["r2_score"][i])
        
r2_df = pd.DataFrame(r2_str_array, columns=rows)

section_path = "chapters/results/figures/"
fname = THESIS_PATH + section_path + "regression_simulated_single_energy_r2.tex"
caption = """
Mean R2-scores for regresson of simulated data using multiple models. 
Error estimates are the standard deviation in results from k-fold cross-validation 
with $K=5$ folds.
"""
label = "tab:regression-simulated-single-energy-r2"
with open(fname, "w") as fp:
    pd.set_option('display.max_colwidth', -1)
    f1_df.to_latex(fp, escape=False, caption=caption, label=label, index=False)


## Double events

### Positions

#### Linear Regression

In [3]:
# Load linear regression experiment
lin_ex_id = "7b74b3cfc586"
lin_ex = load_experiment(lin_ex_id)
print(lin_ex.keys())
#lin_model = tf.keras.models.load_model(repo_root + "models/" + lin_ex_id + ".h5")

lin_metrics = experiment_metrics_to_df(lin_ex)
display(lin_metrics)
lin_means = lin_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
lin_means = lin_means.rename(index={'mean': 'lin_mean', 'std': 'lin_std'})
#display(lin_means)

dict_keys(['loss', 'optimizer', 'experiment_config', 'model_type', 'experiment_name', 'experiment_id', 'datetime', 'history', 'metrics'])


,r2_score,mse,rmse,mae
fold_0,0.368961,0.044194,0.210223,0.171172
fold_1,0.366511,0.044346,0.210586,0.168066
fold_2,0.366158,0.044338,0.210566,0.167999
fold_3,0.354232,0.045276,0.212782,0.167990
fold_4,0.366019,0.044447,0.210824,0.167800


,r2_score,mse,rmse,mae
lin_mean,0.364376,0.044520,0.210996,0.168605
lin_std,0.005796,0.000432,0.001021,0.001438


#### Small dense network

In [4]:
# Load logistic regression experiment
dense_ex_id = "ef55911e49d1"
dense_ex = load_experiment(dense_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + dense_ex_id + ".h5")

dense_metrics = experiment_metrics_to_df(dense_ex)
display(dense_metrics)
dense_means = dense_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
dense_means = dense_means.rename(index={'mean': 'dense_mean', 'std': 'dense_std'})
#display(dense_means)

,r2_score,mse,rmse,mae
fold_0,0.469411,0.037159,0.192766,0.156936
fold_1,0.470370,0.037076,0.192551,0.156727
fold_2,0.469633,0.037100,0.192613,0.156769
fold_3,0.470706,0.037110,0.192639,0.156583
fold_4,0.473899,0.036884,0.192051,0.156205


,r2_score,mse,rmse,mae
dense_mean,0.470804,0.037066,0.192524,0.156644
dense_std,0.001809,0.000106,0.000276,0.000276


#### Small CNN

In [5]:
# Load logistic regression experiment
cnn_ex_id = "cc2654aea019"
cnn_ex = load_experiment(cnn_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + cnn_ex_id + ".h5")

cnn_metrics = experiment_metrics_to_df(cnn_ex)
display(cnn_metrics)
cnn_means = cnn_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
cnn_means = cnn_means.rename(index={'mean': 'cnn_mean', 'std': 'cnn_std'})
#display(cnn_means)

,r2_score,mse,rmse,mae
fold_0,0.469877,0.037126,0.192681,0.156943
fold_1,0.471652,0.036986,0.192318,0.157701
fold_2,0.473943,0.036798,0.191829,0.156764
fold_3,0.473148,0.036939,0.192195,0.156872
fold_4,0.476231,0.036720,0.191624,0.156221


,r2_score,mse,rmse,mae
cnn_mean,0.472970,0.036914,0.192129,0.15690
cnn_std,0.002394,0.000160,0.000415,0.00053


#### Pretrained - VGG
As an additional baseline for performance, we include a pretrained SOTA network
where trained on the ImageNet database.

Due to the size of our detector images (16x16) compared with the size the VGG network is
designed for, we cannot use all layers in the VGG network. This stems from the use of max-pooling
which effectively reduces the image size to half (8x8) each time the input is passed through such a
layer. At some point our input is too small to pass through to the rest of the network.
We therefore cut the network at the point where this becomes an issue.
Alternatively, one could possibly keep the depth but remove max-pooling layers.

In [6]:
# Load logistic regression experiment
pretrained_ex_id = "3c0d1b7bd0ac"
pretrained_ex = load_experiment(pretrained_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + pretrained_ex_id + ".h5")

pretrained_metrics = experiment_metrics_to_df(pretrained_ex)
display(pretrained_metrics)
pretrained_means = pretrained_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
pretrained_means = pretrained_means.rename(index={'mean': 'pretrained_mean', 'std': 'pretrained_std'})
#display(pretrained_means)

,r2_score,mse,rmse,mae
fold_0,0.347832,0.045672,0.213711,0.169636
fold_1,0.361048,0.044728,0.211490,0.167342
fold_2,0.346876,0.045689,0.213750,0.169484
fold_3,0.354489,0.045259,0.212742,0.168016
fold_4,0.373026,0.043955,0.209656,0.165781


,r2_score,mse,rmse,mae
pretrained_mean,0.356654,0.045061,0.212270,0.168052
pretrained_std,0.010787,0.000732,0.001727,0.001599


#### Output
We use the standard deviation in the folds as an error measure, and report the mean classification f1_score.

In [10]:
all_means = pd.DataFrame(
    [
        lin_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        dense_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        cnn_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        pretrained_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']]
    ]
).rename(
    index={
        'lin_mean': 'Linear',
        'dense_mean': 'Dense',
        'cnn_mean': 'Convolutional',
        'pretrained_mean': 'Pretrained VGG16',
    }
)
display(all_means)
all_std = pd.DataFrame(
    [
        lin_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        dense_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        cnn_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        pretrained_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
    ]
).rename(
    index={
        'lin_std': 'Linear',
        'dense_std': 'Dense',
        'cnn_std': 'Convolutional',
        'pretrained_std': 'Pretrained VGG16',
    }
)
display(all_std)

,r2_score,mse,rmse,mae
Linear,0.364376,0.044520,0.210996,0.168605
Dense,0.470804,0.037066,0.192524,0.156644
Convolutional,0.472970,0.036914,0.192129,0.156900
Pretrained VGG16,0.356654,0.045061,0.212270,0.168052


,r2_score,mse,rmse,mae
Linear,0.005796,0.000432,0.001021,0.001438
Dense,0.001809,0.000106,0.000276,0.000276
Convolutional,0.002394,0.000160,0.000415,0.000530
Pretrained VGG16,0.010787,0.000732,0.001727,0.001599


In [15]:
rows = all_means.index
r2_str_array = np.zeros((1, all_means.shape[0]), dtype=object)
for i in range(all_means.shape[0]):
    r2_str_array[0, i] = r"$\underset{{\num{{+- {:.3e} }}  }}{{\num{{ {:.3g} }} }}$".format(
        all_std["r2_score"][i], all_means["r2_score"][i])
        
r2_df = pd.DataFrame(r2_str_array, columns=rows)

section_path = "chapters/results/figures/"
fname = THESIS_PATH + section_path + "regression_simulated_double_position_r2.tex"
caption = """
Mean R2-scores for regresson of simulated data using multiple models. 
Error estimates are the standard deviation in results from k-fold cross-validation 
with $K=5$ folds.
"""
label = "tab:regression-simulated-double-position-r2"
with open(fname, "w") as fp:
    pd.set_option('display.max_colwidth', -1)
    f1_df.to_latex(fp, escape=False, caption=caption, label=label, index=False)


### Energy

#### Linear regression

In [11]:
# Load linear regression experiment
lin_ex_id = "6e600e08e8af"
lin_ex = load_experiment(lin_ex_id)
print(lin_ex.keys())
#lin_model = tf.keras.models.load_model(repo_root + "models/" + lin_ex_id + ".h5")

lin_metrics = experiment_metrics_to_df(lin_ex)
display(lin_metrics)
lin_means = lin_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
lin_means = lin_means.rename(index={'mean': 'lin_mean', 'std': 'lin_std'})
#display(lin_means)

dict_keys(['loss', 'optimizer', 'experiment_config', 'model_type', 'experiment_name', 'experiment_id', 'datetime', 'history', 'metrics'])


,r2_score,mse,rmse,mae
fold_0,0.457530,0.045084,0.212330,0.173310
fold_1,0.451280,0.045791,0.213989,0.174713
fold_2,0.445088,0.046196,0.214932,0.175492
fold_3,0.301623,0.058276,0.241404,0.196951
fold_4,0.419286,0.048458,0.220132,0.179641


,r2_score,mse,rmse,mae
lin_mean,0.414961,0.048761,0.220557,0.180021
lin_std,0.065006,0.005467,0.012013,0.009753


#### Small dense network

In [12]:
# Load dense regression experiment
dense_ex_id = "96cd3707d131"
dense_ex = load_experiment(dense_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + dense_ex_id + ".h5")

dense_metrics = experiment_metrics_to_df(dense_ex)
display(dense_metrics)
dense_means = dense_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
dense_means = dense_means.rename(index={'mean': 'dense_mean', 'std': 'dense_std'})
#display(dense_means)

,r2_score,mse,rmse,mae
fold_0,0.459383,0.044930,0.211967,0.173053
fold_1,0.451509,0.045772,0.213945,0.174737
fold_2,0.447131,0.046025,0.214535,0.175217
fold_3,0.300012,0.058409,0.241680,0.197255
fold_4,0.421377,0.048284,0.219737,0.179411


,r2_score,mse,rmse,mae
dense_mean,0.415882,0.048684,0.220373,0.179935
dense_std,0.066321,0.005576,0.012252,0.009961


#### Small CNN

In [13]:
# Load cnn regression experiment
cnn_ex_id = "f41605cb58b4"
cnn_ex = load_experiment(cnn_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + cnn_ex_id + ".h5")

cnn_metrics = experiment_metrics_to_df(cnn_ex)
display(cnn_metrics)
cnn_means = cnn_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
cnn_means = cnn_means.rename(index={'mean': 'cnn_mean', 'std': 'cnn_std'})
#display(cnn_means)

,r2_score,mse,rmse,mae
fold_0,0.455154,0.045281,0.212794,0.173681
fold_1,0.458946,0.045152,0.212489,0.173719
fold_2,0.458947,0.045042,0.212230,0.173490
fold_3,0.341748,0.054927,0.234365,0.191578
fold_4,0.424079,0.048059,0.219223,0.179010


,r2_score,mse,rmse,mae
cnn_mean,0.427775,0.047692,0.218220,0.178296
cnn_std,0.050267,0.004236,0.009484,0.007782


#### Pretrained - VGG16 

In [14]:
# Load logistic regression experiment
pretrained_ex_id = "9f33b3fc7fff"
pretrained_ex = load_experiment(pretrained_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + pretrained_ex_id + ".h5")

pretrained_metrics = experiment_metrics_to_df(pretrained_ex)
display(pretrained_metrics)
pretrained_means = pretrained_metrics.agg([np.mean, np.std])#.applymap('{:.3f}'.format)
pretrained_means = pretrained_means.rename(index={'mean': 'pretrained_mean', 'std': 'pretrained_std'})
#display(pretrained_means)

,r2_score,mse,rmse,mae
fold_0,0.425208,0.047769,0.218561,0.179244
fold_1,0.435167,0.047136,0.217108,0.175438
fold_2,0.430477,0.047411,0.217741,0.175945
fold_3,0.309650,0.057604,0.240008,0.192079
fold_4,0.419613,0.048432,0.220072,0.177375


,r2_score,mse,rmse,mae
pretrained_mean,0.404023,0.049670,0.222698,0.180016
pretrained_std,0.053075,0.004462,0.009740,0.006903


#### Output

In [15]:
all_means = pd.DataFrame(
    [
        lin_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        dense_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        cnn_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']],
        pretrained_means.iloc[0][['r2_score', 'mse', 'rmse', 'mae']]
    ]
).rename(
    index={
        'lin_mean': 'Linear',
        'dense_mean': 'Dense',
        'cnn_mean': 'Convolutional',
        'pretrained_mean': 'Pretrained VGG16',
    }
)
display(all_means)
all_std = pd.DataFrame(
    [
        lin_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        dense_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        cnn_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
        pretrained_means.iloc[1][['r2_score', 'mse', 'rmse', 'mae']],
    ]
).rename(
    index={
        'lin_std': 'Linear',
        'dense_std': 'Dense',
        'cnn_std': 'Convolutional',
        'pretrained_std': 'Pretrained VGG16',
    }
)
display(all_std)

,r2_score,mse,rmse,mae
Linear,0.414961,0.048761,0.220557,0.180021
Dense,0.415882,0.048684,0.220373,0.179935
Convolutional,0.427775,0.047692,0.218220,0.178296
Pretrained VGG16,0.404023,0.049670,0.222698,0.180016


,r2_score,mse,rmse,mae
Linear,0.065006,0.005467,0.012013,0.009753
Dense,0.066321,0.005576,0.012252,0.009961
Convolutional,0.050267,0.004236,0.009484,0.007782
Pretrained VGG16,0.053075,0.004462,0.009740,0.006903


In [ ]:
rows = all_means.index
r2_str_array = np.zeros((1, all_means.shape[0]), dtype=object)
for i in range(all_means.shape[0]):
    r2_str_array[0, i] = r"$\underset{{\num{{+- {:.3e} }}  }}{{\num{{ {:.3g} }} }}$".format(
        all_std["r2_score"][i], all_means["r2_score"][i])
        
r2_df = pd.DataFrame(r2_str_array, columns=rows)

section_path = "chapters/results/figures/"
fname = THESIS_PATH + section_path + "regression_simulated_double_energy_r2.tex"
caption = """
Mean R2-scores for regresson of simulated data using multiple models. 
Error estimates are the standard deviation in results from k-fold cross-validation 
with $K=5$ folds.
"""
label = "tab:regression-simulated-double-energy-r2"
with open(fname, "w") as fp:
    pd.set_option('display.max_colwidth', -1)
    f1_df.to_latex(fp, escape=False, caption=caption, label=label, index=False)
